In [783]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, accuracy_score)

In [784]:
df = pd.read_csv('data/weather_data_hcm_[2017-2022]_preprocess.csv')

In [785]:
df.head()

,Time,Day,Month,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather
0,00:00,1,1,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,46.6,Partly cloudy
1,03:00,1,1,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,26.9,Partly cloudy
2,06:00,1,1,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,29.2,Partly cloudy
3,09:00,1,1,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,66.7,Partly cloudy
4,12:00,1,1,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,73.0,Partly cloudy


In [786]:
label = df['Weather']
features = df

In [787]:
#remove data
features = features[:-1]
label = label[1:]

In [788]:
data = features
data['Label'] = label.values

/tmp/ipykernel_883394/1655283038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Label'] = label.values


In [789]:
RAINY_LABEL = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain',
             'Patchy light rain with thunder','Patchy rain possible','Heavy rain','Heavy rain at times',
             'Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast','Torrential rain shower']
RAINY_MONTH = [5,6,7,8,9,10,11]
MORNING_HOUR = [6,9,12,15]

In [790]:
for i, value in enumerate(data.Label):
    if value in RAINY_LABEL:
        data.Label[i] = 'yes'
    else:
        data.Label[i] = 'no'

/tmp/ipykernel_883394/4224172725.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Label[i] = 'no'
/data4/thang/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_883394/4224172725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Label[i] = 'yes'


In [791]:
# for i, value in enumerate(data.Time):
#     hour = value.split(':')[0]
#     data.Time[i] = hour

In [792]:
for i, value in enumerate(data.Weather):
    if value in RAINY_LABEL:
        data.Weather[i] = 'rainy'
    else:
        data.Weather[i] = 'sunny'

/tmp/ipykernel_883394/812369229.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Weather[i] = 'sunny'
/tmp/ipykernel_883394/812369229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Weather[i] = 'rainy'


In [793]:
# for i, value in enumerate(data.Month):
#     if int(value) in RAINY_MONTH:
#         data.Month[i] = 'rainy_month'
#     else:
#         data.Month[i] = 'sunny_month'

In [794]:
# remove columns
# drop_columns = ['Day', 'Year', 'Humidity', 'Pressure']
drop_columns = []
data = data.drop(drop_columns, axis=1)
data.head()

,Time,Day,Month,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather,Label
0,00:00,1,1,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,46.6,sunny,no
1,03:00,1,1,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,26.9,sunny,no
2,06:00,1,1,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,29.2,sunny,no
3,09:00,1,1,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,66.7,sunny,no
4,12:00,1,1,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,73.0,sunny,no


In [795]:
def convert_category2numeric(data, columns_name):
    for col in columns_name:
        data = pd.concat((data, pd.get_dummies(data[col], prefix=col)), axis=1)
        data = data.drop([col], axis=1)
    return data

In [796]:
data = convert_category2numeric(data, ['Time', 'Weather', 'Month'])
data = pd.concat((data, pd.get_dummies(data['Label'], prefix='Label', drop_first=True)), axis=1)
data = data.drop(['Label'], axis=1)
data.head()

,Day,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,...,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Label_yes
0,1,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,...,0,0,0,0,0,0,0,0,0,0
1,1,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,...,0,0,0,0,0,0,0,0,0,0
2,1,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,...,0,0,0,0,0,0,0,0,0,0
3,1,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,...,0,0,0,0,0,0,0,0,0,0
4,1,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,...,0,0,0,0,0,0,0,0,0,0


In [800]:
all_columns = data.columns.to_list()
Y_colums = ['Label_yes']
X_colums = [x for x in all_columns if x not in Y_colums]

In [801]:
train_set, test_set = train_test_split(data, test_size=0.2)

In [802]:
Xtrain = train_set[X_colums]
ytrain = train_set[Y_colums]

# building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: inf
         Iterations 8


/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Label_yes   No. Observations:                11408
Model:                          Logit   Df Residuals:                    11377
Method:                           MLE   Df Model:                           30
Date:                Mon, 25 Jul 2022   Pseudo R-squ.:                     inf
Time:                        09:21:08   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Day               0.0027      0.003      0.866      0.386      -0.003       0.009
Year             -0.0874      0.018     -4.770      0.000      -0.123      -0.051
Humidity          0.0061      0.004      1.452      0.146      -0.002       0.014
Dew point         0.1328      0.025      5.282      0.000       0.084       0.182
Temperature       0.0308      0.020      1.537      0.124      -0.008       0.070
Rain              0.3444      0.028     12.283      0.000       0.289       0.399
Cloud             0.0377      0.001     27.015      0.000       0.035       0.040
Pressure         -0.0535      0.019     -2.808      0.005      -0.091      -0.016
Wind             -0.0615      0.026     -2.367      0.018      -0.112      -0.011
Gust              0.0668      0.019      3.593      0.000       0.030       0.103
Dir               0.0002      0.000      0.508      0.611      -0.001       0.001
Time_00:00       38.9574   5.81e+06   6.71e-06      1.000   -1.14e+07    1.14e+07
Time_03:00       38.9532   5.82e+06    6.7e-06      1.000   -1.14e+07    1.14e+07
Time_06:00       39.0336   5.81e+06   6.72e-06      1.000   -1.14e+07    1.14e+07
Time_09:00       39.3919   5.81e+06   6.78e-06      1.000   -1.14e+07    1.14e+07
Time_12:00       40.0770    5.8e+06   6.91e-06      1.000   -1.14e+07    1.14e+07
Time_15:00       40.0300   5.81e+06   6.89e-06      1.000   -1.14e+07    1.14e+07
Time_18:00       38.9895    5.8e+06   6.72e-06      1.000   -1.14e+07    1.14e+07
Time_21:00       39.3339   5.81e+06   6.77e-06      1.000   -1.14e+07    1.14e+07
Weather_rainy   157.7781        nan        nan        nan         nan         nan
Weather_sunny   156.9882        nan        nan        nan         nan         nan
Month_1          25.5326   5.23e+06   4.89e-06      1.000   -1.02e+07    1.02e+07
Month_2          25.7360   5.23e+06   4.92e-06      1.000   -1.02e+07    1.02e+07
Month_3          25.5246   5.23e+06   4.88e-06      1.000   -1.02e+07    1.02e+07
Month_4          26.1884   5.23e+06   5.01e-06      1.000   -1.02e+07    1.02e+07
Month_5          26.3571   5.23e+06   5.04e-06      1.000   -1.02e+07    1.02e+07
Month_6          26.6222   5.23e+06   5.09e-06      1.000   -1.02e+07    1.02e+07
Month_7          26.2995   5.23e+06   5.03e-06      1.000   -1.02e+07    1.02e+07
Month_8          26.2107   5.23e+06   5.02e-06      1.000   -1.02e+07    1.02e+07
Month_9          26.6896   5.23e+06   5.11e-06      1.000   -1.02e+07    1.02e+07
Month_10         26.9317   5.23e+06   5.15e-06      1.000   -1.02e+07    1.02e+07
Month_11         26.5500   5.23e+06   5.08e-06      1.000   -1.02e+07    1.02e+07
Month_12         26.1240   5.23e+06      5e-06      1.000   -1.02e+07    1.02e+07
=================================================================================
"""

In [803]:
Xtest = test_set[all_columns[:-1]]
ytest = test_set[all_columns[-1]]

yhat = log_reg.predict(Xtest)
prediction = list(map(round, yhat))
cm = confusion_matrix(ytest, prediction) 
print ("Confusion Matrix : \n", cm)

# accuracy score of the model
print('Test accuracy = ', accuracy_score(ytest, prediction))

Confusion Matrix : 
 [[1614  236]
 [ 258  744]]
Test accuracy =  0.8267882187938289


In [804]:
log_reg.save('log_model.pickle')

In [805]:
log_model = sm.load('log_model.pickle')

In [806]:
a = dict(Xtest.iloc[[0]])

In [818]:
inputs = {}
for k,v in a.items():
    inputs[k] = v.values[0]

In [824]:
x = pd.DataFrame([inputs])

In [825]:
x

,Day,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
0,14,2017,94.0,24.0,26.0,0.0,22.0,1010.0,9.0,10.0,...,0,0,0,0,0,0,0,0,1,0


In [827]:

y = log_model.predict(x)
y = list(map(round, y))

In [833]:
inputs

{'Day': 14,
 'Year': 2017,
 'Humidity': 94.0,
 'Dew point': 24.0,
 'Temperature': 26.0,
 'Rain': 0.0,
 'Cloud': 22.0,
 'Pressure': 1010.0,
 'Wind': 9.0,
 'Gust': 10.0,
 'Dir': 73.5,
 'Time_00:00': 0,
 'Time_03:00': 0,
 'Time_06:00': 1,
 'Time_09:00': 0,
 'Time_12:00': 0,
 'Time_15:00': 0,
 'Time_18:00': 0,
 'Time_21:00': 0,
 'Weather_rainy': 0,
 'Weather_sunny': 1,
 'Month_1': 0,
 'Month_2': 0,
 'Month_3': 0,
 'Month_4': 0,
 'Month_5': 0,
 'Month_6': 0,
 'Month_7': 0,
 'Month_8': 0,
 'Month_9': 0,
 'Month_10': 0,
 'Month_11': 1,
 'Month_12': 0}

In [837]:
import json

for k,v in a.items():
    inputs[k] = 0
with open('intput.json', 'w') as f:
    json.dump(inputs, f)